In [1]:
import os
import shutil
import yaml
import logging
from copy import deepcopy
import regex
import emoji
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cleaning
from datetime import datetime
from time import sleep
from pathlib import Path

In [49]:
import importlib
import sys

spec_src = importlib.util.spec_from_file_location(
    'src', 
    '../../__init__.py')
m = importlib.util.module_from_spec(spec_src)
sys.modules[spec_src.name] = m
spec_src.loader.exec_module(m)

from src import utils

In [51]:
logger = utils.get_logger('cleaning-corpes', datetime.now(), desc='Removing unnecessary columns, adding an id column')

In [20]:
""" Reload changed modules without restarting kernel """
importlib.reload(utils)

<module 'src.utils' from '/home/rimov/Documents/Code/NLP/lin-que-dropping/src/analysis/cleaning/../../utils.py'>

In [141]:
gen_conf = utils.get_config()
conf = utils.get_config('c')

In [22]:
extracted_path = utils.get_save_path('e', 'corpes', is_test=False, lang='es')
extracted_folders = ['2021-12-06/volitional', '2022-01-12/stative', '2022-01-12/epistemic']

In [23]:
es_conjug_path = utils.get_project_root()/gen_conf['file_paths']['verb_conjug']

es_verbs_conjug = pd.read_excel(es_conjug_path)
es_verbs_conjug.head(2)

,verb_type,verb,indicativo,imperativo,subjuntivo,gerundio,gerundio_compuesto,infinitivo,infinitivo_compuesto,participio_pasado
0,Stative,ver,veía visto verías vi vimos verían ves v...,vean ve vea veamos ved,veáis visto vieras vieren viesen veas vi...,viendo,visto,ver,visto,visto
1,Stative,jurar,jurarán juramos jurarías jurabas juraría ...,jurad jura juren jure juremos,jurare jurareis jurase jurara juraren jur...,jurando,jurado,jurar,jurado,jurado


### Organize Files

In [30]:
cleaned_path = utils.get_save_path('c', 'corpes', lang='es')
cleaned_folders = extracted_folders

for f in extracted_folders:
    try:
        utils.make_dir(cleaned_path/f)
        logger.info(f'Creating directory for file organization at: {extracted_path/f}')
    except:
        continue

In [32]:
logger.info('Creating separate verb directories')

for v in es_verbs_conjug['verb'].values:
    for folder in cleaned_folders:
        try:
            utils.make_dir(cleaned_path/folder, v)
        except:
            logger.exception(f'Couldn\'t create directory for "{v}"')

In [33]:
verb_pat = conf['corpes']['verb_pat']
    
for folder in extracted_folders:
    ep = extracted_path/folder
    cp = cleaned_path/folder
    
    logger.info(f'Copying files from: \n{ep} \nto: \n{cp}')
    
    for file in ep.iterdir():
        match = regex.search(verb_pat, file.name)
        v = match['keyword'].lower().strip()
        
        try:
            new_name = file.name.replace(' ', '')
            shutil.copyfile(file, cp/v/new_name)
        except FileNotFoundError as fe:
            logger.exception(fe.args)
            logger.exception(f'Probably no directory for verb: {v}')
            continue

In [34]:
# Remove any empty directories which were created while looping through verbs
for folder in cleaned_folders:
    utils.remove_empty_dirs(cleaned_path/folder)

['confirmar', 'dudar', 'parecer', 'afirmar', 'temer', 'mencionar', 'pensar', 'suspirar', 'contar', 'prometer', 'jurar', 'admitir', 'asegurar', 'comprobar', 'mostrar', 'conseguir', 'recomendar', 'lamentar', 'apostar', 'predecir', 'saber', 'acordar', 'sentir', 'adivinar', 'suponer', 'gritar', 'negar', 'reclamar', 'ojala', 'lograr', 'ver', 'recordar', 'decir', 'demostrar', 'entender', 'creer', 'considerar', 'confesar', 'responder', 'imaginar', 'prever']
['dudar', 'suplicar', 'parecer', 'solicitar', 'temer', 'pensar', 'jurar', 'rogar', 'esperar', 'querer', 'comprobar', 'recomendar', 'lamentar', 'apostar', 'predecir', 'saber', 'desear', 'pedir', 'acordar', 'sentir', 'adivinar', 'suponer', 'negar', 'ojala', 'ordenar', 'recordar', 'entender', 'creer', 'imaginar', 'mandar', 'prever']
['confirmar', 'suplicar', 'solicitar', 'afirmar', 'mencionar', 'pensar', 'suspirar', 'contar', 'prometer', 'jurar', 'admitir', 'rogar', 'asegurar', 'esperar', 'querer', 'mostrar', 'conseguir', 'desear', 'pedir', '

### Column Names and Adding Ids

In [153]:
""" Reload changed modules without restarting kernel """
importlib.reload(utils)

<module 'src.utils' from '/home/rimov/Documents/Code/NLP/lin-que-dropping/src/analysis/cleaning/../../utils.py'>

In [154]:
id_count = 0

In [155]:
save_folder = utils.get_str_datetime_now()

In [158]:
for folder in cleaned_folders:
    folder_path = ((cleaned_path/folder).parent)/save_folder
    utils.make_dir(folder_path)
    
    for verb in (cleaned_path/folder).iterdir():
        verb_path = folder_path/verb.name
        utils.make_dir(verb_path)
        
        for file in verb.iterdir():
            data = utils.get_csv('corpes', file, sep='\t', lineterminator=None)
            data = data.rename(columns=cleaning.standardize_col_name)
            data = cleaning.assign_ids(data, id_count)
            
            display(data.head(10))
            break
        break
    break
            
#             utils.save_csv(verb_path, data, file.stem)
            
#             # Update used id count
#             id_count = id_count + len(data)
#             print(id_count)

,id,bibliografia,autor,titulo,fecha,criterio,bloque,medio,soporte,tema,pais,zona,tipologia,notas,concordancia
0,126826,"Moro, Javier: Mi pecado. Barcelona: Espasa, 2018.","Moro, Javier",Mi pecado,2018,Primera edición,Ficción,Escrito,Libro,Novela,España,NaN,Ficción,Premio Primavera 2018,En 1937 Conchita supo que las cosas en Madrid ...
1,126827,"Lozano Garbala, David: Desconocidos. Barcelona...","Lozano Garbala, David",Desconocidos,2018,Primera edición,Ficción,Escrito,Libro,Novela,España,NaN,Ficción,Premio Edebé de literatura juvenil,"«Habla», suplica Lara, quieta como una estatua..."
2,126828,"Lozada, Victoria: La buena nutrición. La salud...","Lozada, Victoria",La buena nutrición. La salud empieza en tu lis...,2018,Primera edición,No ficción,Escrito,Libro,Salud,España,NaN,Divulgación,<NA>,Cuando pasaron por la zona en donde las sirena...
3,126829,"Gómez-Jurado, Juan: Reina Roja. Barcelona: Pen...","Gómez-Jurado, Juan",Reina Roja,2018,Primera edición,Ficción,Escrito,Libro,Novela,España,NaN,Ficción,<NA>,-Por favor. Puede ser nuestra última oportunid...
4,126830,"Gómez-Jurado, Juan: Reina Roja. Barcelona: Pen...","Gómez-Jurado, Juan",Reina Roja,2018,Primera edición,Ficción,Escrito,Libro,Novela,España,NaN,Ficción,<NA>,"-¿Qué tiene pensado, Sandra? Si lo sabes, tien..."
5,126831,"«El padre de Gabriel Cruz: ""Si lo que quieren ...",<NA>,La Vanguardia,2018,Primera edición,No ficción,Escrito,Prensa,"Actualidad, ocio y vida cotidiana",España,NaN,Noticia,<NA>,"Ángel Cruz, el padre del niño de 8 años desapa..."
6,126832,"Congostrina, Alfonso L.: «La Guardia Urbana de...","Congostrina, Alfonso L.",El País,2018,Primera edición,No ficción,Escrito,Prensa,"Política, economía y justicia",España,NaN,Noticia,<NA>,Pero el que acaparó la mayor atención fue Hati...
7,126833,"Hernández, Marisol: «Malestar en parte del Gob...","Hernández, Marisol",El Mundo,2018,Primera edición,No ficción,Escrito,Prensa,"Política, economía y justicia",España,NaN,Noticia,<NA>,"señalado otras y ha dicho algo muy importante,..."
8,126834,"Cervera, César: «La terrible infancia de Feder...","Cervera, César",ABC,2018,Primera edición,No ficción,Escrito,Prensa,"Ciencias sociales, creencias y pensamiento",España,NaN,Divulgación,<NA>,Federico suplicó a su padre e incluso le pidió...
9,126835,"Cervera, César: «Las razones por las que los c...","Cervera, César",ABC,2018,Primera edición,No ficción,Escrito,Prensa,"Ciencias sociales, creencias y pensamiento",España,NaN,Divulgación,<NA>,La batalla que decidió el destino del país sor...
